In [1]:
# ! pip install pandas

Сетка n на m  
Одна клетка = 0 (пустая клетка) или 1 (препятствие)  
Начальная клетка - A, конечная клетка (заключительное состояние) - B

**Действия:**  
переход в одну из соседних по стороне клеток

**Динамика среды:**  
- переход выполняется, если клетка пуста
- не выполняется (агент остается в той же клетке), если препятствие или зашли за границы сетки
- переходы из конечной клетки не выполняются
- вознаграждение: -1 за перемещение

In [2]:
# Сетка
grid = [
    [0, 0, 0, 1, 0, 0],
    [0, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 0]
]

# Размерности сетки
n, m = len(grid), len(grid[0])

# Конечная клетка - заключительное состояние
B = (n-1, m-1)

# Действия (изменения координат)
actions = [
    (-1, 0), (0, 1), (1, 0), (0, -1)
]

### 4.1 ДП. Оценивание стратегии

Стратегия: переход в одну из соседних клеток с равной вероятностью $\frac{1}{4}$  

Построить оценку функции ценности для состояний при использовании этой стратегии  

Обесценивание не применяется ($\gamma = 1$)  
Т.о. ценность состояния = отрицательное мат. ожидание длины пути до заключительного состояния при использовании данной стратегии  
Недостаток - если из каких-то состояний не достижимо заключительное состояние, ценность будет бесконечно увеличиваться, алгоритм не сойдётся к $\nu$

In [3]:
# Стратегия
base_pi = [[(1/4, 1/4, 1/4, 1/4) for _ in range(m)] for _ in range(n)]

# Точность оценки
theta = 1e-5

# Алгоритм итеративного оценивания стратегии для оценивания V
def dp_value_function(pi, V):
    iter_count = 0
    while True:
        iter_count += 1
        delta = 0
        for i in range(n):
            for j in range(m):
                v = V[i][j]
                V[i][j] = 0
                # Не считаем ценность для закл. состояния и клеток с препятствиями
                if grid[i][j] == 1 or (i, j) == B:
                    continue
                for action_i, (di, dj) in enumerate(actions):
                    i_next, j_next = i + di, j + dj
                    # Переходим в соседнюю клетку
                    if 0 <= i_next < n and 0 <= j_next < m and grid[i_next][j_next] == 0:
                        V[i][j] += pi[i][j][action_i] * (-1 + V[i_next][j_next])
                    # Остаёмся в текущей клетке
                    else:
                        V[i][j] += pi[i][j][action_i] * (0 + v)   # Правильно ли обновлять оценку состояния через пред. оценку этого состояния?
                delta = max(delta, abs(v - V[i][j]))
        if delta < theta:
            break

    print(f'Точность: {theta}')
    print(f'Количество итераций: {iter_count}')
    return V

V = [[0] * m for _ in range(n)]
V = dp_value_function(base_pi, V)

import pandas as pd
pd.set_option('display.precision', 3)
print('Сетка:')
print(pd.DataFrame(grid))
print('Оценка функции ценности состояний:')
print(pd.DataFrame(V))

Точность: 1e-05
Количество итераций: 2164
Сетка:
   0  1  2  3  4  5
0  0  0  0  1  0  0
1  0  1  1  0  0  0
2  0  0  0  0  0  0
3  0  0  1  0  0  0
4  0  0  0  1  1  0
Оценка функции ценности состояний:
         0        1        2        3        4        5
0 -189.454 -192.454 -193.454    0.000 -104.064 -102.300
1 -184.454    0.000    0.000 -109.808 -103.828  -98.536
2 -177.454 -165.788 -140.788 -113.789  -98.904  -86.480
3 -179.121 -176.121    0.000 -101.654  -87.519  -59.000
4 -180.788 -180.454 -181.454    0.000    0.000    0.000


### 4.3 ДП. Итерация по стратегиям

Найдём оптимальную стратегию для задачи выше

In [4]:
pi = base_pi.copy()
V = [[0] * m for _ in range(n)]
epsilon = 0.001

def dp_optimal_strategy(pi, V):
    while True:
        # Оценивание стратегии
        V = dp_value_function(pi, V)

        # Улучшение стратегии
        policy_stable = True
        for i in range(n):
            for j in range(m):
                if grid[i][j] == 1 or (i, j) == B:
                    continue

                old_action = pi[i][j]

                q_values = []
                for (di, dj) in actions:
                    i_next, j_next = i + di, j + dj
                    if 0 <= i_next < n and 0 <= j_next < m and grid[i_next][j_next] == 0:
                        q = -1 + V[i_next][j_next]
                    else:
                        q = 0 + V[i][j]
                    q_values.append(q)

                best_action = q_values.index(max(q_values))

                # Эпсилон-мягкая стратегия
                new_pi = [epsilon / len(actions)] * len(actions)
                new_pi[best_action] = 1 - epsilon + (epsilon / len(actions))
                pi[i][j] = tuple(new_pi)

                if old_action != pi[i][j]:
                    policy_stable = False

        if policy_stable:
            return pi, V

pi_opt, V_opt = dp_optimal_strategy(pi, V)

print('Оценка функции ценности состояний для опт. стратегии:')
print(pd.DataFrame(V))

print('Оптимальная стратегия:')
print(pd.DataFrame(pi_opt))

Точность: 1e-05
Количество итераций: 2164
Точность: 1e-05
Количество итераций: 33711
Точность: 1e-05
Количество итераций: 7
Точность: 1e-05
Количество итераций: 1
Оценка функции ценности состояний для опт. стратегии:
       0       1       2      3      4      5
0 -9.007 -10.008 -11.008  0.000 -5.004 -4.004
1 -8.007   0.000   0.000 -5.004 -4.004 -3.003
2 -7.006  -6.005  -5.004 -4.004 -3.003 -2.002
3 -8.007  -7.006   0.000 -3.003 -2.002 -1.001
4 -9.007  -8.007  -9.007  0.000  0.000  0.000
Оптимальная стратегия:
                                      0                                     1  \
0  (0.00025, 0.00025, 0.99925, 0.00025)  (0.00025, 0.00025, 0.00025, 0.99925)   
1  (0.00025, 0.00025, 0.99925, 0.00025)              (0.25, 0.25, 0.25, 0.25)   
2  (0.00025, 0.99925, 0.00025, 0.00025)  (0.00025, 0.99925, 0.00025, 0.00025)   
3  (0.99925, 0.00025, 0.00025, 0.00025)  (0.99925, 0.00025, 0.00025, 0.00025)   
4  (0.99925, 0.00025, 0.00025, 0.00025)  (0.99925, 0.00025, 0.00025, 0.00025)  

#### Новая модель:

Пусть у свободной клетки будет значение $x$:  
$0 \le x < 1$ - "степень пробуксовки" агента в клетке  
x = вероятность остаться в клетке при попытке выхода из неё

$P[(i', j') | (i, j), a] = 1 - x$  
$P[(i, j) | (i, j), a] = x$

Будем давать вознаграждение -5 за действие в клетке с $0 < x < 1$, если агент остался в клетке
(попробовать давать вознаграждение зависящее от x)

Пусть $x$ имеет точность до 1 знака после запятой  
Будем хранить в сетке целые значения $10x$

In [5]:
# Сетка
grid = [
    [0, 0, 1, 1, 0, 0],
    [0, 5, 5, 5, 0, 0],
    [0, 5, 8, 5, 0, 10],
    [0, 5, 5, 5, 0, 0],
    [0, 0, 0, 0, 0, 0]
]

# Размерности сетки
n, m = len(grid), len(grid[0])

# Конечная клетка - заключительное состояние
B = (n-1, m-1)

In [6]:
# Стратегия
base_pi = [[(1/4, 1/4, 1/4, 1/4) for _ in range(m)] for _ in range(n)]

# Точность оценки
theta = 1e-5

# Алгоритм итеративного оценивания стратегии для оценивания V
def dp_value_function(pi, V):
    iter_count = 0
    while True:
        iter_count += 1
        delta = 0
        for i in range(n):
            for j in range(m):
                v = V[i][j]
                V[i][j] = 0
                # Не считаем ценность для закл. состояния и клеток с препятствиями
                if grid[i][j] == 10 or (i, j) == B:
                    continue
                for action_i, (di, dj) in enumerate(actions):
                    i_next, j_next = i + di, j + dj
                    # Переходим в соседнюю клетку
                    if 0 <= i_next < n and 0 <= j_next < m and grid[i_next][j_next] != 10:
                        V[i][j] += pi[i][j][action_i] * (1 - grid[i][j] / 10) * (-1 + V[i_next][j_next])
                        V[i][j] += pi[i][j][action_i] * (grid[i][j] / 10) * (-5 + v)
                    # Остаёмся в текущей клетке
                    else:
                        V[i][j] += pi[i][j][action_i] * (0 + v)
                delta = max(delta, abs(v - V[i][j]))
        if delta < theta:
            break

    print(f'Точность: {theta}')
    print(f'Количество итераций: {iter_count}')
    return V

V = [[0] * m for _ in range(n)]
V = dp_value_function(base_pi, V)

import pandas as pd
pd.set_option('display.precision', 3)
print('Сетка:')
print(pd.DataFrame(grid))
print('Оценка функции ценности состояний:')
print(pd.DataFrame(V))

Точность: 1e-05
Количество итераций: 1918
Сетка:
   0  1  2  3  4   5
0  0  0  1  1  0   0
1  0  5  5  5  0   0
2  0  5  8  5  0  10
3  0  5  5  5  0   0
4  0  0  0  0  0   0
Оценка функции ценности состояний:
         0        1        2        3        4        5
0 -483.089 -481.769 -473.310 -456.747 -441.056 -437.883
1 -482.410 -485.908 -476.748 -451.208 -425.538 -432.710
2 -475.231 -478.706 -472.564 -421.800 -373.176    0.000
3 -461.579 -457.120 -429.002 -366.254 -269.191 -135.595
4 -449.386 -435.193 -396.072 -321.021 -197.737    0.000


In [7]:
pi = base_pi.copy()
V = [[0] * m for _ in range(n)]
epsilon = 0.001

def dp_optimal_strategy(pi, V):
    while True:
        # Оценивание стратегии
        V = dp_value_function(pi, V)

        # Улучшение стратегии
        policy_stable = True
        for i in range(n):
            for j in range(m):
                if grid[i][j] == 10 or (i, j) == B:
                    continue

                old_action = pi[i][j]

                q_values = []
                for (di, dj) in actions:
                    i_next, j_next = i + di, j + dj
                    if 0 <= i_next < n and 0 <= j_next < m and grid[i_next][j_next] != 10:
                        q = (1 - grid[i][j] / 10) * (-1 + V[i_next][j_next])
                        q += (grid[i][j] / 10) * (-5 + V[i][j])
                    else:
                        q = 0 + V[i][j]
                    q_values.append(q)

                best_action = q_values.index(max(q_values))

                # Эпсилон-мягкая стратегия
                new_pi = [epsilon / len(actions)] * len(actions)
                new_pi[best_action] = 1 - epsilon + (epsilon / len(actions))
                pi[i][j] = tuple(new_pi)

                if old_action != pi[i][j]:
                    policy_stable = False

        if policy_stable:
            return pi, V

pi_opt, V_opt = dp_optimal_strategy(pi, V)

print('Оценка функции ценности состояний для опт. стратегии:')
print(pd.DataFrame(V))

print('Оптимальная стратегия:')
print(pd.DataFrame(pi_opt))

Точность: 1e-05
Количество итераций: 1918
Точность: 1e-05
Количество итераций: 78
Точность: 1e-05
Количество итераций: 55
Оценка функции ценности состояний для опт. стратегии:
       0       1       2       3      4      5
0 -9.015  -9.126  -8.125  -6.566 -5.009 -6.008
1 -8.015 -14.018 -14.132 -10.012 -4.008 -5.008
2 -7.012 -13.021 -30.017  -9.015 -3.005  0.000
3 -6.010 -10.012  -9.015  -8.006 -2.003 -1.001
4 -5.008  -4.008  -3.006  -2.003 -1.001  0.000
Оптимальная стратегия:
                                      0                                     1  \
0  (0.00025, 0.00025, 0.99925, 0.00025)  (0.00025, 0.99925, 0.00025, 0.00025)   
1  (0.00025, 0.00025, 0.99925, 0.00025)  (0.00025, 0.00025, 0.00025, 0.99925)   
2  (0.00025, 0.00025, 0.99925, 0.00025)  (0.00025, 0.00025, 0.00025, 0.99925)   
3  (0.00025, 0.00025, 0.99925, 0.00025)  (0.00025, 0.00025, 0.99925, 0.00025)   
4  (0.00025, 0.99925, 0.00025, 0.00025)  (0.00025, 0.99925, 0.00025, 0.00025)   

                                

Проблема переноса опыта: используя текущую модель, можно обучить агента проходить конкретную сетку, но не перенести накопленный опыт на другую сетку

#### Новая модель:

Пусть состояние среды будет не положение агента $(i, j)$ в сетке, а  
$s$ = (характеристика ($x$) текущей клетки, характеристики соседних по стороне клеток, смещение от цели ($\Delta i, \Delta j$))